In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
#Load the dataset
data = pd.read_csv('Churn_Modelling.csv')
data.head()
data.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [3]:
data = data.drop(['RowNumber','CustomerId','Surname'], axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
#Encode categorical variables
one_hot_encoder_gen = OneHotEncoder(sparse_output=False)
gen_encoder=one_hot_encoder_gen.fit_transform(data[['Gender']])
gen_encoder 

one_hot_encoder_geo = OneHotEncoder(sparse_output=False)
geo_encoder=one_hot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [6]:
##these are the column names
one_hot_encoder_geo.get_feature_names_out(['Geography'])
one_hot_encoder_gen.get_feature_names_out(['Gender'])

array(['Gender_Female', 'Gender_Male'], dtype=object)

In [7]:
gen_encoded_df = pd.DataFrame(gen_encoder, columns=one_hot_encoder_gen.get_feature_names_out(['Gender']))
gen_encoded_df
geo_encoded_df = pd.DataFrame(geo_encoder, columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [8]:
#drop the Grography and Gender column
data = data.drop(['Geography','Gender'], axis=1)
data


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,42,2,0.00,1,1,1,101348.88,1
1,608,41,1,83807.86,1,0,1,112542.58,0
2,502,42,8,159660.80,3,1,0,113931.57,1
3,699,39,1,0.00,2,0,0,93826.63,0
4,850,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0
9996,516,35,10,57369.61,1,1,1,101699.77,0
9997,709,36,7,0.00,1,0,1,42085.58,1
9998,772,42,3,75075.31,2,1,0,92888.52,1


In [9]:
data = pd.concat([data,gen_encoded_df,geo_encoded_df], axis=1)
data

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Gender_Female,Gender_Male,Geography_France,Geography_Germany,Geography_Spain
0,619,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,1.0,0.0,0.0
1,608,41,1,83807.86,1,0,1,112542.58,0,1.0,0.0,0.0,0.0,1.0
2,502,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,1.0,0.0,0.0
3,699,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,1.0,0.0,0.0
4,850,43,2,125510.82,1,1,1,79084.10,0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,0.0,1.0,1.0,0.0,0.0
9996,516,35,10,57369.61,1,1,1,101699.77,0,0.0,1.0,1.0,0.0,0.0
9997,709,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,1.0,0.0,0.0
9998,772,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0,1.0,0.0


In [10]:
#Save the encoders

with open('onehot_encoder_gender.pkl', 'wb') as file:
     pickle.dump(one_hot_encoder_gen, file)
with open('onehot_encoder_geo.pkl', 'wb') as file:
     pickle.dump(one_hot_encoder_geo, file)


In [11]:
#divide the data into dependent and independent
X = data.drop('Exited', axis=1)
y = data['Exited']

In [12]:
#Split the data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# #Scale the features
scaler= StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train

array([[ 0.35649971, -0.6557859 ,  0.34567966, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.29493847, -0.3483691 , ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213, -1.41636539, -0.69539349, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -0.08535128, -1.38944225, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.3900109 ,  1.03972843, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  1.15059039, -1.38944225, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [13]:
X_train.shape, X_test.shape

((8000, 13), (2000, 13))

In [14]:
#save the scaled file in pickle form
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

### ANN implementation

In [27]:
import tensorflow 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,  Dropout
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [26]:
! pip install tensorflow==2.12.0

  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached grpcio-1.65.4-cp311-cp311-win_amd64.whl.metadata (3.4 kB)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp311-cp311-win_amd64.whl.metadata (14 kB)
  Using cached ml_dtypes-0.4.0-cp311-cp311-win_amd64.whl.metadata (20 kB)
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
  Using cached google_auth-2.33.0-py2.py3-none-any.whl.metadata (4.7 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none

In [21]:
y_test.shape

(2000,)

In [28]:
#Model building
model=Sequential()


#hidden layer1
model.add(Dense(100, activation='sigmoid', input_shape=(13,)))
model.add(Dropout(0.3))
#hidden layer2
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
#output layer
model.add(Dense(1, activation='sigmoid'))



In [29]:
import tensorflow
opt =tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss= tensorflow.keras.losses.BinaryCrossentropy
loss

keras.losses.BinaryCrossentropy

In [30]:
#Compile the model - to do forward and backwrd propagation
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [31]:
#Set up the tensorboard to visualise the logs that we have when training the entire model
log_dir = "logs/fit/" +datetime.datetime.now().strftime("%Y%m%d - %H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [32]:
#Setup Early stopping
early_stopping_callback = EarlyStopping(monitor = "val_loss", patience = 10, restore_best_weights=True)

In [33]:
#Train the model - apply the callsback here
history=model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test),
                   callbacks=[tensorflow_callback,early_stopping_callback])

Epoch 1/100
250/250 [==============================] - 4s 8ms/step - loss: 0.4852 - accuracy: 0.7918 - val_loss: 0.4212 - val_accuracy: 0.8035
Epoch 2/100
250/250 [==============================] - 1s 6ms/step - loss: 0.4587 - accuracy: 0.7961 - val_loss: 0.4403 - val_accuracy: 0.8035
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4496 - accuracy: 0.8030 - val_loss: 0.4086 - val_accuracy: 0.8205
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4441 - accuracy: 0.8087 - val_loss: 0.3985 - val_accuracy: 0.8370
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4362 - accuracy: 0.8148 - val_loss: 0.3889 - val_accuracy: 0.8370
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4254 - accuracy: 0.8179 - val_loss: 0.3819 - val_accuracy: 0.8425
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4205 - accuracy: 0.8175 - val_loss: 0.3643 - val_accuracy: 0.8470

In [34]:
#load the entire tensorboard
model.save("model.h5")


In [35]:
#load the entire tensorboard extension
%load_ext tensorboard

In [40]:
%tensorboard --logdir logs/fit/ --port=6006

Reusing TensorBoard on port 6006 (pid 16616), started 0:00:08 ago. (Use '!kill 16616' to kill it.)

In [ ]:
##Load the pickle files-there are three pickle files that we have here.Let's do this in prediction.ipynb file.

